In [ ]:
!pip install phevaluator

In [ ]:
from phevaluator import evaluate_cards
import random

suits = ['d','s','c','h']
ranks = ['A','2','3','4','5','6','7','8','9','T','J','Q','K']
cards = []
for r in ranks:
    for s in suits:
        cards.append(r+s)

def simulate(hand, table, players):
    hands = []
    deck = random.sample(cards,len(cards)) #shuffle the deck
    hand = hand[:]
    table = table[:]

    full = table + hand
    deck = list(filter(lambda x: x not in full, deck))

    #deal cards to players
    for i in range(players):
        hn = []
        hn.append(deck[0])
        deck = deck[1:]
        hn.append(deck[0])
        deck = deck[1:]
        hands.append(hn)

    #flop, turn, river
    while len(table) < 5:
        card = deck.pop(0)
        table.append(card)
        full.append(card)
    my_hand_rank = evaluate_cards(full[0],full[1],full[2],full[3],full[4],full[5],full[6])

    result_state = 0  # 'WIN' - assume win until proven otherwise
    for check_hand in hands:
        all_cards = table + check_hand
        opponent = evaluate_cards(all_cards[0],all_cards[1],all_cards[2],all_cards[3],all_cards[4],all_cards[5],all_cards[6])
        # from the definition of the library we use for hand evaluation, larger evaluations correspond to less strong hands
        #so, the game is won by the player with the smallest hand evaluation
        if opponent < my_hand_rank:
            result_state = 1  # 'LOSE' - there is no comeback from losing, stop comparing
            break
        if opponent == my_hand_rank:
            result_state = 2  # 'SPLIT' - on split, we still might lose to another player
    return result_state


def monte_carlo(hand, table, players=2, samples=10000):
    dist = [0,0,0]

    for i in range(samples):
        outcome = simulate(hand, table, players)
        dist[outcome] += 1
    return list(map(lambda x: x/samples, dist))


In [ ]:
monte_carlo(['Ah','Ad'], ['Kc','2d','9s'])

In [ ]:
class MCSamplingPlayer(AIPlayer):
    def __init__(self, balance) -> None:
        super().__init__(balance)

    def place_bet(self, observed_env) -> int:  # AI will call every time
        """
        A Strategy implemented with human heuristics.
        """
        if "k" in observed_env.valid_actions():
            action = "k"
        else:
            action = "c"

        card_str = [str(card) for card in self.hand]
        community_cards = [str(card) for card in observed_env.community_cards]

        isDealer = self == observed_env.get_player(observed_env.dealer_button_position)
        checkAllowed = "k" in observed_env.valid_actions()

        action = self.get_action(
            card_str,
            community_cards,
            observed_env.total_pot_balance,
            observed_env.get_highest_current_bet(),
            observed_env.BIG_BLIND,
            self.player_balance,
            isDealer,
            checkAllowed,
        )

        self.process_action(action, observed_env)  # use voice activation
        return action

    def get_action(
        self,
        card_str,
        community_cards,
        total_pot_balance,
        highest_current_bet,
        BIG_BLIND,
        player_balance,
        isDealer,
        checkAllowed,
    ):
        equity = monte_carlo(card_str, community_cards)
        # fold, check / call, raise
        fold_prob = 1.0 - (equity + equity / 2.0)
        if fold_prob < 0 :
          fold_prob = 0.0
        np_strategy = np.abs(np.array([fold_prob, equity, equity / 2.0]))
        np_strategy = np_strategy / np.sum(np_strategy)  # normalize

        if highest_current_bet == 0:  # no bet placed
            if isDealer:  # If you are the dealer, raise more of the time
                strategy = {
                    "k": np_strategy[0],
                    f"b{min(max(BIG_BLIND, int(total_pot_balance / 3)), player_balance)}": np_strategy[
                        2
                    ],
                    f"b{min(total_pot_balance, player_balance)}": np_strategy[1],
                }
            else:
                strategy = {
                    "k": equity,
                    f"b{min(total_pot_balance, player_balance)}": 1 - equity,
                }

        else:  # if there is a bet already
            if checkAllowed:
                strategy = {
                    "k": np_strategy[0],
                    f"b{min(int(1.5 * highest_current_bet), player_balance)}": np_strategy[1],
                    f"b{min(2 * highest_current_bet, player_balance)}": np_strategy[2],
                }
            else:
                if highest_current_bet == player_balance:
                    strategy = {
                        "f": np_strategy[0],
                        "c": np_strategy[1] + np_strategy[2],
                    }
                else:
                    strategy = {
                        "f": np_strategy[0],
                        "c": np_strategy[1],
                        f"b{min(2 * highest_current_bet, player_balance)}": np_strategy[2],
                    }

        # renormalize the strategy in case of duplicates
        total = sum(strategy.values())
        for key in strategy:
            strategy[key] /= total
        action = getAction(strategy)
        return action
